# Creating SQL Databases

### See Your Tables

* https://www.mysql.com/products/workbench/
* https://dbeaver.io/


### Schemas

https://dbdiagram.io/home

Creating SQL DBs in a few ways!! 

1. Pandas' `.to_sql` method
2. Manually via cursor and `INSERT INTO`
3. On an AWS RDB

In [1]:
import pandas as pd

In [2]:
# importing our sample data

customers = pd.read_csv('data/customers.csv')
order_details = pd.read_csv('data/orderdetails.csv')
orders = pd.read_csv('data/orders.csv')
products = pd.read_csv('data/products.csv')

display(customers.head(), order_details.head())

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",NaN,Nantes,NaN,44000,France,1370.0,21000
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,NaN,Las Vegas,NV,83030,USA,1166.0,71800
2,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Level 3,Melbourne,Victoria,3004,Australia,1611.0,117300
3,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",NaN,Nantes,NaN,44000,France,1370.0,118200
4,121,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Erling Skakkes gate 78,NaN,Stavern,NaN,4110,Norway,1504.0,81700


,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber
0,10100,S18_1749,30,136.00,3
1,10100,S18_2248,50,55.09,2
2,10100,S18_4409,22,75.46,4
3,10100,S24_3969,49,35.29,1
4,10101,S18_2325,25,108.06,4


## Using Pandas' `.to_sql()` method

In [5]:
from sqlalchemy import create_engine

# must start with 'sqlite:///' for a relative path
engine = create_engine('sqlite:///sales.db', echo=True) 
# echo determines whether actions are output

In [6]:
# as a function
def create_sql_table(df, table_name, engine):
    df.to_sql(table_name, con=engine, if_exists='replace')

In [7]:
create_sql_table(customers, 'customers', engine)

2021-09-16 14:08:16,397 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("customers")
2021-09-16 14:08:16,398 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-16 14:08:16,399 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("customers")
2021-09-16 14:08:16,399 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-16 14:08:16,401 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-16 14:08:16,401 INFO sqlalchemy.engine.Engine 
CREATE TABLE customers (
	"index" BIGINT, 
	"customerNumber" BIGINT, 
	"customerName" TEXT, 
	"contactLastName" TEXT, 
	"contactFirstName" TEXT, 
	phone TEXT, 
	"addressLine1" TEXT, 
	"addressLine2" TEXT, 
	city TEXT, 
	state TEXT, 
	"postalCode" TEXT, 
	country TEXT, 
	"salesRepEmployeeNumber" FLOAT, 
	"creditLimit" BIGINT
)


2021-09-16 14:08:16,402 INFO sqlalchemy.engine.Engine [no key 0.00044s] ()
2021-09-16 14:08:16,404 INFO sqlalchemy.engine.Engine CREATE INDEX ix_customers_index ON customers ("index")
2021-09-16 14:08:16,405 INFO sqlalchemy.engine

In [8]:
create_sql_table(order_details, 'order_details', engine)

2021-09-16 14:08:16,631 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("order_details")
2021-09-16 14:08:16,632 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-16 14:08:16,633 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("order_details")
2021-09-16 14:08:16,634 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-09-16 14:08:16,636 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-16 14:08:16,637 INFO sqlalchemy.engine.Engine 
CREATE TABLE order_details (
	"index" BIGINT, 
	"orderNumber" BIGINT, 
	"productCode" TEXT, 
	"quantityOrdered" BIGINT, 
	"priceEach" FLOAT, 
	"orderLineNumber" BIGINT
)


2021-09-16 14:08:16,637 INFO sqlalchemy.engine.Engine [no key 0.00054s] ()
2021-09-16 14:08:16,639 INFO sqlalchemy.engine.Engine CREATE INDEX ix_order_details_index ON order_details ("index")
2021-09-16 14:08:16,640 INFO sqlalchemy.engine.Engine [no key 0.00061s] ()
2021-09-16 14:08:16,641 INFO sqlalchemy.engine.Engine COMMIT
2021-09-16 14:08:16,645 INFO sqlalchemy.engine.Engin

### Re-reading in our data

In [11]:
result = engine.execute('SELECT * FROM customers;')
pd.DataFrame(result.fetchall(), columns = result.keys())

2021-09-16 14:12:14,206 INFO sqlalchemy.engine.Engine SELECT * FROM customers;
2021-09-16 14:12:14,207 INFO sqlalchemy.engine.Engine [raw sql] ()


,index,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",None,Nantes,None,44000,France,1370.0,21000
1,1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,None,Las Vegas,NV,83030,USA,1166.0,71800
2,2,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Level 3,Melbourne,Victoria,3004,Australia,1611.0,117300
3,3,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",None,Nantes,None,44000,France,1370.0,118200
4,4,121,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Erling Skakkes gate 78,None,Stavern,None,4110,Norway,1504.0,81700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,117,486,Motor Mint Distributors Inc.,Salazar,Rosa,2155559857,11328 Douglas Av.,None,Philadelphia,PA,71270,USA,1323.0,72600
118,118,487,Signal Collectibles Ltd.,Taylor,Sue,4155554312,2793 Furth Circle,None,Brisbane,CA,94217,USA,1165.0,60300
119,119,489,"Double Decker Gift Stores, Ltd",Smith,Thomas,(171) 555-7555,120 Hanover Sq.,None,London,None,WA1 1DP,UK,1501.0,43300
120,120,495,Diecast Collectables,Franco,Valarie,6175552555,6251 Ingle Ln.,None,Boston,MA,51003,USA,1188.0,85100


In [10]:
result = engine.execute('SELECT * FROM order_details;')
pd.DataFrame(result.fetchall(), columns = result.keys())

2021-09-16 14:11:58,953 INFO sqlalchemy.engine.Engine SELECT * FROM order_details;
2021-09-16 14:11:58,953 INFO sqlalchemy.engine.Engine [raw sql] ()


,index,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber
0,0,10100,S18_1749,30,136.00,3
1,1,10100,S18_2248,50,55.09,2
2,2,10100,S18_4409,22,75.46,4
3,3,10100,S24_3969,49,35.29,1
4,4,10101,S18_2325,25,108.06,4
...,...,...,...,...,...,...
2991,2991,10425,S24_2300,49,127.79,9
2992,2992,10425,S24_2840,31,31.82,5
2993,2993,10425,S32_1268,41,83.79,11
2994,2994,10425,S32_2509,11,50.32,6


## AWS RDS (Relational Database System)

Note: this database just lives on AWS, you can do this same process on a local database!

Pros and cons of these vs `.to_sql()`:
* You can configure your DB more professionally
* Fairly manual process

Setting up a database instance: https://aws.amazon.com/getting-started/hands-on/create-mysql-db/ 

**It is extremely important that you read all the instructions in the linked page so everything is set up correctly!!!**

In [12]:
import mysql.connector
from mysql.connector import errorcode

In [13]:
# connecting to your database
cnx = mysql.connector.connect(
    host = 'database-2.cpizore09tpz.us-east-2.rds.amazonaws.com',
    user = 'admin',
    passwd = 'testing123'
)

# cursor so we can interact with the db
cursor = cnx.cursor()

In [15]:
cursor.execute("DROP DATABASE sales;")

DatabaseError: 1008 (HY000): Can't drop database 'sales'; database doesn't exist

In [16]:
# creating your database

db_name = 'sales'

def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

In [17]:
try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exist.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)
        
cursor.close()
cnx.close()

Database sales does not exist.
Database sales created successfully.


In [18]:
# creating tables

TABLES = {}
TABLES['customers'] = (
        "CREATE TABLE customers ("
        " customerNumber int(11) NOT NULL,"
        " customerName varchar(64) NOT NULL,"
        " contactLastName varchar(16) NOT NULL,"
        " contactFirstName varchar(16) NOT NULL,"
        " phone varchar(16) NOT NULL,"
        " addressLine1 varchar(64) NOT NULL,"
        " addressLine2 varchar(64) NOT NULL,"
        " city varchar(64) NOT NULL,"
        " state varchar(64) NOT NULL,"
        " postalCode int(10) NOT NULL,"
        " country varchar(64) NOT NULL,"
        " salesRepEmployeeNumber int(11) NOT NULL,"
        " creditLimit int(11) NOT NULL) ENGINE=InnoDB"
)



In [19]:
customers.head()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",NaN,Nantes,NaN,44000,France,1370.0,21000
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,NaN,Las Vegas,NV,83030,USA,1166.0,71800
2,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Level 3,Melbourne,Victoria,3004,Australia,1611.0,117300
3,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",NaN,Nantes,NaN,44000,France,1370.0,118200
4,121,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Erling Skakkes gate 78,NaN,Stavern,NaN,4110,Norway,1504.0,81700


In [20]:
print(TABLES['customers'])

CREATE TABLE customers ( customerNumber int(11) NOT NULL, customerName varchar(64) NOT NULL, contactLastName varchar(16) NOT NULL, contactFirstName varchar(16) NOT NULL, phone varchar(16) NOT NULL, addressLine1 varchar(64) NOT NULL, addressLine2 varchar(64) NOT NULL, city varchar(64) NOT NULL, state varchar(64) NOT NULL, postalCode int(10) NOT NULL, country varchar(64) NOT NULL, salesRepEmployeeNumber int(11) NOT NULL, creditLimit int(11) NOT NULL) ENGINE=InnoDB


In [21]:
TABLES

{'customers': 'CREATE TABLE customers ( customerNumber int(11) NOT NULL, customerName varchar(64) NOT NULL, contactLastName varchar(16) NOT NULL, contactFirstName varchar(16) NOT NULL, phone varchar(16) NOT NULL, addressLine1 varchar(64) NOT NULL, addressLine2 varchar(64) NOT NULL, city varchar(64) NOT NULL, state varchar(64) NOT NULL, postalCode int(10) NOT NULL, country varchar(64) NOT NULL, salesRepEmployeeNumber int(11) NOT NULL, creditLimit int(11) NOT NULL) ENGINE=InnoDB'}

In [25]:
# connecting to your database
cnx = mysql.connector.connect(
    host = 'database-2.cpizore09tpz.us-east-2.rds.amazonaws.com',
    user = 'admin',
    passwd = 'testing123',
    database = 'sales'
)

# cursor so we can interact with the db
cursor = cnx.cursor()

In [26]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

cursor.close()
cnx.close()

Creating table customers: already exists.


### Inserting Data

In [27]:
customers.columns

Index(['customerNumber', 'customerName', 'contactLastName', 'contactFirstName',
       'phone', 'addressLine1', 'addressLine2', 'city', 'state', 'postalCode',
       'country', 'salesRepEmployeeNumber', 'creditLimit'],
      dtype='object')

In [28]:
insert = ("INSERT INTO customers (customerNumber, "
"customerName, contactLastName, contactFirstName, phone, "
"addressLine1, addressLine2, city, state, postalCode, "
"country, salesRepEmployeeNumber, creditLimit) VALUES "
"(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")

In [29]:
insert

'INSERT INTO customers (customerNumber, customerName, contactLastName, contactFirstName, phone, addressLine1, addressLine2, city, state, postalCode, country, salesRepEmployeeNumber, creditLimit) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'

In [30]:
customers = customers.dropna()
customers['salesRepEmployeeNumber'] = customers['salesRepEmployeeNumber'].astype(int)
customers = customers.applymap(str)

In [31]:
customers.head()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
2,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Level 3,Melbourne,Victoria,3004,Australia,1611,117300
15,151,Muscle Machine Inc,Young,Jeff,2125557413,4092 Furth Circle,Suite 400,NYC,NY,10022,USA,1286,138500
20,168,American Souvenirs Inc,Franco,Keith,2035557845,149 Spinnaker Dr.,Suite 101,New Haven,CT,97823,USA,1286,0
27,181,Vitachrome Inc.,Frick,Michael,2125551500,2678 Kingston Rd.,Suite 101,NYC,NY,10022,USA,1286,76400
46,240,giftsbymail.co.uk,Bennett,Helen,(198) 555-8888,Garden House,Crowther Way 23,Cowes,Isle of Wight,PO31 7PJ,UK,1501,93900


In [32]:
first = tuple(customers.iloc[0].values)
first

('114',
 'Australian Collectors, Co.',
 'Ferguson',
 'Peter',
 '03 9520 4555',
 '636 St Kilda Road',
 'Level 3',
 'Melbourne',
 'Victoria',
 '3004',
 'Australia',
 '1611',
 '117300')

In [33]:
# connecting to your database
cnx = mysql.connector.connect(
    host = 'database-2.cpizore09tpz.us-east-2.rds.amazonaws.com',
    user = 'admin',
    passwd = 'testing123',
    database = 'sales'
)

# cursor so we can interact with the db
cursor = cnx.cursor()

# this is the insert!
cursor.execute(insert, first)

# commiting changes and closing the connection
cnx.commit()
cursor.close()
cnx.close()

In [36]:
data = [tuple(x) for x in customers.to_numpy()]
data

[('114',
  'Australian Collectors, Co.',
  'Ferguson',
  'Peter',
  '03 9520 4555',
  '636 St Kilda Road',
  'Level 3',
  'Melbourne',
  'Victoria',
  '3004',
  'Australia',
  '1611',
  '117300'),
 ('151',
  'Muscle Machine Inc',
  'Young',
  'Jeff',
  '2125557413',
  '4092 Furth Circle',
  'Suite 400',
  'NYC',
  'NY',
  '10022',
  'USA',
  '1286',
  '138500'),
 ('168',
  'American Souvenirs Inc',
  'Franco',
  'Keith',
  '2035557845',
  '149 Spinnaker Dr.',
  'Suite 101',
  'New Haven',
  'CT',
  '97823',
  'USA',
  '1286',
  '0'),
 ('181',
  'Vitachrome Inc.',
  'Frick',
  'Michael',
  '2125551500',
  '2678 Kingston Rd.',
  'Suite 101',
  'NYC',
  'NY',
  '10022',
  'USA',
  '1286',
  '76400'),
 ('240',
  'giftsbymail.co.uk',
  'Bennett',
  'Helen ',
  '(198) 555-8888',
  'Garden House',
  'Crowther Way 23',
  'Cowes',
  'Isle of Wight',
  'PO31 7PJ',
  'UK',
  '1501',
  '93900'),
 ('276',
  "Anna's Decorations, Ltd",
  "O'Hara",
  'Anna',
  '02 9936 8555',
  '201 Miller Street',
  

In [37]:
# connecting to your database
cnx = mysql.connector.connect(
    host = 'database-2.cpizore09tpz.us-east-2.rds.amazonaws.com',
    user = 'admin',
    passwd = 'testing123',
    database = 'sales'
)

# cursor so we can interact with the db
cursor = cnx.cursor()

cursor.executemany(insert, data)

cnx.commit()
cursor.close()
cnx.close()

In [39]:
# connecting to your database
cnx = mysql.connector.connect(
    host = 'database-2.cpizore09tpz.us-east-2.rds.amazonaws.com',
    user = 'admin',
    passwd = 'testing123',
    database = 'sales'
)
cursor = cnx.cursor()

cursor.execute("SELECT * FROM customers;")
pd.DataFrame(cursor.fetchall(), columns=[x[0] for x in cursor.description]).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   customerNumber          10 non-null     int64 
 1   customerName            10 non-null     object
 2   contactLastName         10 non-null     object
 3   contactFirstName        10 non-null     object
 4   phone                   10 non-null     object
 5   addressLine1            10 non-null     object
 6   addressLine2            10 non-null     object
 7   city                    10 non-null     object
 8   state                   10 non-null     object
 9   postalCode              10 non-null     int64 
 10  country                 10 non-null     object
 11  salesRepEmployeeNumber  10 non-null     int64 
 12  creditLimit             10 non-null     int64 
dtypes: int64(4), object(9)
memory usage: 1.1+ KB


You can either use the above method to read in your data, or use `pd.read_sql`